In [1]:
#Hey I've added this so we can see properly SQL Views from Spark (Simon)
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pyspark
import numpy as np
from pyspark.sql import SQLContext
sc = pyspark.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
import pandas as pd
import re
from pyspark.sql import SparkSession
from snorkel.labeling import PandasLFApplier,LFAnalysis
spark = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()
pd_dev = pd.read_csv("review_dev_labelled.csv",header=0, index_col=0)
df_dev = spark.createDataFrame(pd_dev)

In [4]:
#Try to apply Labeling function
import pyspark.sql.functions as F
from snorkel.labeling import LabelModel
from snorkel.labeling.apply.spark import SparkLFApplier
import pandas as pd
import numpy as np
#from snorkel.labeling import ,LFAnalysis
from snorkel.labeling import LFAnalysis
from pyspark.sql import Row
from snorkel.labeling.lf import labeling_function
#from snorkel.labeling.lf.nlp_spark import spark_nlp_labeling_function
from snorkel.preprocess import preprocessor

ABSTAIN = -1
NEGATIVE = 0
POSITIVE = 1

# LFs

In [5]:
from snorkel.labeling import LabelingFunction


@labeling_function()
def short_comment(x):
    """Negative comments are often short, such as 'foods were insane'"""
    return NEGATIVE if len(x.text.split()) < 20 else ABSTAIN

@labeling_function()
def keyword_disgusting(x):
    """Negative comments are often short, such as 'foods were insane'"""
    return POSITIVE if "disgusting" in x.text.lower() else ABSTAIN

@labeling_function()
def keyword_cardboard(x):
    """Negative comments are often short, such as 'foods were insane'"""
    return POSITIVE if "cardboard" in x.text.lower() else ABSTAIN

@labeling_function()
def regex_home(x):
    return POSITIVE if re.search(r"home.*about", x.text, flags=re.I) else ABSTAIN

@labeling_function()
def regex_regular(x):
    return NEGATIVE if re.search(r"regular*", x.text, flags=re.I) else ABSTAIN
@labeling_function()
def regex_recomend(x):
    return POSITIVE if re.search(r"not.recommend*", x.text, flags=re.I) else ABSTAIN

@labeling_function()
def Keyword_gem(x):
    return NEGATIVE if "gem" in x.text.lower() else ABSTAIN

@labeling_function()
def Keyword_perfect(x):
    return NEGATIVE if "perfect" in x.text.lower() else ABSTAIN

@labeling_function()
def Keyword_avoid(x):
    return POSITIVE if "avoid" in x.text.lower() else ABSTAIN

@labeling_function()
def regex_over_coocked(x):
    return POSITIVE if re.search(r"over.cooked*", x.text, flags=re.I) else ABSTAIN

@labeling_function()
def Keyword_overpriced(x):
    return POSITIVE if "overpriced" in x.text.lower() else ABSTAIN

@labeling_function()
def long_comments(x):
    """Negative comments are often short, such as 'foods were insane'"""
    return POSITIVE if len(x.text.split()) > 350 else ABSTAIN

@labeling_function()
def key_good(x):
    """Negative comments are often short, such as 'foods were insane'"""
    return POSITIVE if "bad" in x.text.lower() else ABSTAIN



In [6]:
lfs = [short_comment,long_comments,keyword_disgusting,keyword_cardboard,regex_home,
       Keyword_perfect,Keyword_avoid,regex_over_coocked, Keyword_overpriced,key_good]
applier = SparkLFApplier(lfs)
L_dev = applier.apply(df_dev.rdd)
g_label =np.array(df_dev.select('label').collect())
LFAnalysis(L_dev, lfs).lf_summary(g_label)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
short_comment,0,[0],0.048,0.002,0.002,17,7,0.708333
long_comments,1,[1],0.042,0.022,0.014,17,4,0.809524
keyword_disgusting,2,[1],0.006,0.002,0.000,3,0,1.000000
keyword_cardboard,3,[1],0.004,0.002,0.000,2,0,1.000000
regex_home,4,[1],0.014,0.006,0.002,5,2,0.714286
Keyword_perfect,5,[0],0.100,0.020,0.020,36,14,0.720000
Keyword_avoid,6,[1],0.016,0.006,0.002,6,2,0.750000
regex_over_coocked,7,[1],0.006,0.004,0.002,3,0,1.000000
Keyword_overpriced,8,[1],0.010,0.004,0.000,5,0,1.000000
key_good,9,[1],0.074,0.012,0.004,29,8,0.783784


# Augmentation

In [7]:
pip install utils

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install utils-py

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz


     |████████████████████████████████| 12.0 MB 727 kB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.0-py3-none-any.whl size=12019121 sha256=da70f878878423ed2d0ada92a9f8d579f54418c0303f7e72c6217e65090c69cc
  Stored in directory: /home/faculty/.cache/pip/wheels/1d/bc/94/171b09b7fcce517723f40606754e5b7374770cc39290e092bf
Successfully built en-core-web-sm
Note: you may need to restart the kernel to use updated packages.


In [10]:
#pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz


In [11]:
import nltk
from nltk.corpus import wordnet as wn
from snorkel.augmentation import transformation_function
import spacy
from snorkel.preprocess.nlp import SpacyPreprocessor
import en_core_web_sm
spacy = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)

import utils
#from utils import preview_tfs
from snorkel.augmentation import RandomPolicy
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /home/faculty/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
#Define TFs

def get_synonym(word, pos=None):
    """Get synonym for word given its part-of-speech (pos)."""
    synsets = wn.synsets(word, pos=pos)
    # Return None if wordnet has no synsets (synonym sets) for this word and pos.
    if synsets:
        words = [lemma.name() for lemma in synsets[0].lemmas()]
        if words[0].lower() != word.lower():  # Skip if synonym is same as word.
            # Multi word synonyms in wordnet use '_' as a separator e.g. reckon_with. Replace it with space.
            return words[0].replace("_", " ")
        
def replace_token(spacy_doc, idx, replacement):
    """Replace token in position idx with replacement."""
    return " ".join([spacy_doc[:idx].text, replacement, spacy_doc[1 + idx :].text])

@transformation_function(pre=[spacy])
def replace_verb_with_synonym(x):
    # Get indices of verb tokens in sentence.
    verb_idxs = [i for i, token in enumerate(x.doc) if token.pos_ == "VERB"]
    if verb_idxs:
        # Pick random verb idx to replace.
        idx = np.random.choice(verb_idxs)
        synonym = get_synonym(x.doc[idx].text, pos="v")
        # If there's a valid verb synonym, replace it. Otherwise, return None.
        if synonym:
            x.text = replace_token(x.doc, idx, synonym)
            return x
        
@transformation_function(pre=[spacy])
def replace_noun_with_synonym(x):
    # Get indices of noun tokens in sentence.
    noun_idxs = [i for i, token in enumerate(x.doc) if token.pos_ == "NOUN"]
    if noun_idxs:
        # Pick random noun idx to replace.
        idx = np.random.choice(noun_idxs)
        synonym = get_synonym(x.doc[idx].text, pos="n")
        # If there's a valid noun synonym, replace it. Otherwise, return None.
        if synonym:
            x.text = replace_token(x.doc, idx, synonym)
            return x


@transformation_function(pre=[spacy])
def replace_adjective_with_synonym(x):
    # Get indices of adjective tokens in sentence.
    adjective_idxs = [i for i, token in enumerate(x.doc) if token.pos_ == "ADJ"]
    if adjective_idxs:
        # Pick random adjective idx to replace.
        idx = np.random.choice(adjective_idxs)
        synonym = get_synonym(x.doc[idx].text, pos="a")
        # If there's a valid adjective synonym, replace it. Otherwise, return None.
        if synonym:
            x.text = replace_token(x.doc, idx, synonym)
            return x

In [13]:
tfs = [replace_verb_with_synonym,replace_noun_with_synonym,replace_adjective_with_synonym]

In [14]:
#
from snorkel.augmentation import RandomPolicy
random_policy = RandomPolicy(
    len(tfs), sequence_length=2, n_per_original=2, keep_original=True
)

In [15]:
from snorkel.augmentation import PandasTFApplier
tf_applier = PandasTFApplier(tfs,random_policy)


In [16]:
df_train_augmented = tf_applier.apply(pd_dev)
Y_train_augmented = df_train_augmented["label"].values


100%|██████████| 500/500 [00:23<00:00, 20.95it/s]


In [19]:
print(f"Original training set size: {pd_dev.size}")
print(f"Augmented training set size: {df_train_augmented.size}")
print(np.shape(pd_dev))
print(np.shape(df_train_augmented))

Original training set size: 6000
Augmented training set size: 12912
(500, 12)
(1076, 12)


In [18]:
# I checked the rows and the replacement works.
#df_train_augmented.tail()
pd.options.display.max_rows = 999
df_train_augmented.tail(100)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,index_1,index_2,label
454,fbKoXozkAKDxoWg8eyQk9Q,1,20/09/2007 18:06,0,_lwxHEwCgd-q-WS5P7ksZA,4,"This is actually one place, I will agree with ...",1,6Z9T9BAhyg1_rs7xE7PwIQ,1.250000e+12,454,0
455,2Cmpz9lgr-wfOLAwgUf79g,0,07/10/2016 20:12,0,59LpRh6cjrgYVSoMIiR4Uw,1,Definitely understaffed. 30 minutes wait with ...,1,3uArj1R2P1CJQeEfP9qdpQ,1.480000e+12,455,1
455,2Cmpz9lgr-wfOLAwgUf79g,0,07/10/2016 20:12,0,59LpRh6cjrgYVSoMIiR4Uw,1,Definitely understaffed. 30 minutes wait with ...,1,3uArj1R2P1CJQeEfP9qdpQ,1.480000e+12,455,1
455,2Cmpz9lgr-wfOLAwgUf79g,0,07/10/2016 20:12,0,59LpRh6cjrgYVSoMIiR4Uw,1,Definitely understaffed. 30 minutes wait with ...,1,3uArj1R2P1CJQeEfP9qdpQ,1.480000e+12,455,1
456,FYa3GUro-ezTM9zvKiocFw,0,30/09/2014 04:30,0,AZTqpAhF5j-dzr3g0roSGA,5,I get the steak kabob pita and it is awesome e...,0,#NAME?,1.500000e+12,456,0
457,2iTsRqUsPGRH1li1WVRvKQ,1,04/03/2016 05:44,1,AhBH45zBEXAXxI7YqFGFpQ,5,First time ever coming here and definitely won...,1,c9yABxGxddckfYxH-sVqbA,1.550000e+11,457,0
458,jrFB2o5Gk2aRw22oNGxJKg,0,14/03/2011 23:15,0,cV5cNEELz3CkZD7trJLRxw,2,Maybe I went on an off day but I was not happy...,0,UuUZklYPZd9VhYv9X92Xbw,9.710000e+11,458,1
458,jrFB2o5Gk2aRw22oNGxJKg,0,14/03/2011 23:15,0,cV5cNEELz3CkZD7trJLRxw,2,Maybe I went on an off day but I was not happy...,0,UuUZklYPZd9VhYv9X92Xbw,9.710000e+11,458,1
459,D-2t7fHQu6hNLjjfRHSZaA,0,28/03/2016 16:20,0,rzd5B4HsC578ZjdTAiy3cQ,1,Food is terrible. Short rib was like a piece o...,0,5DKvj-mCYrTa8hOGOoj4vw,1.250000e+12,459,1
460,xdD3EXvF_p9WUiwwKpol_w,0,05/04/2016 21:51,0,AhhCiQL0F9Luu0XmZ0SDfw,5,Everything was absolutely awesome. Jody (barte...,0,z-liXDJrx3Rcdo0MDS-gxw,4.550000e+11,460,0
